In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_ollama import ChatOllama

model_name="ministral-3:3b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [30]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        query="SELECT COUNT(*) FROM Artist"  # Granite could not create a valid query
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [31]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [32]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        tools = [sql_query] # internal users get access to all tools
        request = request.override(tools=tools) 
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [33]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [34]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database? use tools")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I currently do not have access to tools to directly query the database for the number of artists. Could you clarify if you're referring to a specific database (e.g., a music database, a platform like Spotify, Apple Music, or a custom system)? If so, please provide more details so I can assist you better!


In [38]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database? use tools")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

There are **275 artists** in the database.
